In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


`trulens_eval.tru_feedback` is deprecated, use `trulens_eval.feedback` instead.
`trulens_eval.tru_app` is deprecated, use `trulens_eval.app` instead.


In [3]:
from trulens_eval import tru_feedback, Feedback, Query
from trulens_eval.utils.llama import WithFeedbackFilterNodes

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [4]:
type(index).__bases__

(llama_index.indices.base.BaseIndex,)

In [5]:
type(query_engine.retriever._vector_store).__bases__

(llama_index.vector_stores.types.VectorStore,)

In [6]:
dir(query_engine.synthesize.__func__.__closure__)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [7]:
from trulens_eval import Tru, Query, Feedback
from trulens_eval import feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
Starting dashboard ...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.1.10.101:8501 .


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# For aggregation,
import numpy as np

# Construct feedback functions.

hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import PineconeVectorStore
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
import pinecone

# Pinecone configuration if using pinecone.
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

pinecone_index = pinecone.Index(index_name="llmdemo")

pinecone_store = PineconeVectorStore(pinecone_index)
index = VectorStoreIndex.from_vector_store(pinecone_store)
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
retriever_filtered = WithFeedbackFilterNodes.of_index_retriever(retriever, feedback=f_qs_relevance, threshold = 0.5)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

query_engine_filtered = RetrieverQueryEngine(
    retriever=retriever_filtered,
)

In [ ]:
res = query_engine.query("Who is Shayak?")

In [ ]:
pp.pprint(res.response)
pp.pprint(res.source_nodes)

In [ ]:
res = query_engine_filtered.query("Who is Shayak?")

In [ ]:
pp.pprint(res.response)
pp.pprint(res.source_nodes)

In [ ]:
l = tru.Llama(engine=query_engine, feedbacks=feedbacks, chain_id="default")
l_filtered = tru.Llama(engine=query_engine_filtered, feedbacks=feedbacks, chain_id="context filtered")

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("Who is Shayak?")

In [ ]:
# record.dict()

In [ ]:
res, record = l_filtered.query_with_record("Who is Shayak?")